## 수집 환경 설정

In [ ]:
import pandas as pd
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import re
import random
import requests

In [ ]:
#크롬드라이브 연결
driver = webdriver.Chrome('C:\chromedriver_win32\chromedriver.exe')

##스크래핑 함수 정의
1. 카페 검색 결과 내에서 원하는 페이지로 이동
2. 텍스트 띄어쓰기 제거
3. 게시물에서 원하는 정보 수집

In [ ]:
# 카페 검색 결과 내에서 원하는 페이지로 이동
def move_page(page):
    url='https://section.cafe.naver.com/ca-fe/home/search/articles?q="제로 웨이스트"&p='+str(page)+'&od=1&pr=7&ps=2021.04.01&pe=2021.04.21'
    return url

# 텍스트 띄어쓰기 제거
def no_space(text):
    text1=re.sub('\n','',text)
    return text1

# 게시물에서 원하는 정보 수집
def get_data(driver):
    driver.switch_to.frame("cafe_main")

    html = driver.page_source
    soup = bs(html, 'html.parser')

    Date=soup.find('span',{'class':'date'}).text

    try:
        Like=soup.find('em',{'class':'u_cnt _count'}).text
    except:
        Like='0'
    Like=int(Like)

    try:
        Content = soup.find('div', {'class': 'article_viewer'}).text.strip()
    except:
        Content=''
    Content=no_space(Content)

    try:
        Hashtag = soup.find('a', {'class': 'tag_link'}).text.strip()
    except:
        Hashtag = ''

    data=[Date,Like,Content,Hashtag]
    return data

## 스크래핑 진행
수집되는 정보 : 게시물 URL, 게시물 게시일자, 좋아요 수, 본문내용, 해시태그

In [ ]:
result=[]
hrefs_list=[]

for page in range(48):

    # 검색결과 페이지 내 게시물 URL 수집
    html = driver.page_source
    soup = bs(html, 'html.parser')
    rink=soup.find_all('div',{'class':'detail_area'})
    hrefs=[div.find('a')['href'] for div in rink]

    for i in range(10):
        hrefs_list.append(hrefs[i])

    # 검색결과 내 페이지 내 게시물별로 접속
    url=move_page(page)
    driver.get(url)
    time.sleep(1)
    article_list = driver.find_elements_by_css_selector('a.item_subject')

    # 게시물 접속 후 원하는 정보 수집
    for article in article_list:
        article.click()
        time.sleep(2)

        change_tab = driver.window_handles[-1]
        driver.switch_to.window(change_tab)

        try:
            data=get_data(driver)
            result.append(data)
        except:
            pass

        driver.close()

        change_tab = driver.window_handles[-1]
        driver.switch_to.window(change_tab)

# 수집된 게시물 정보 + 게시물 URL 저장
df=pd.DataFrame(result,columns=['Date','Like','Content','Hashtag'])
df.insert(0,"URL",pd.Series(hrefs_list),True)
df.to_csv('제로웨이스트0401_0421.csv',index=False, encoding="utf-8-sig")